###### Title: Browse multiple molecules
# Description: browsing multiple molecules, each of which has multiple conformations
Derived from Gerg's code by Malitha 
(Updated: 17th June 2017)

In [2]:
import os
import py3Dmol
from rdkit import Chem
from rdkit.Chem import AllChem
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display
from time import time

# Definition: Default Rendering Style

In [3]:
global myGlobalStyle
molSize_3d = (400, 400)
myGlobalStyle = 'stick' # default drawing type for 3d structures # line cross stick cartoon sphere
bgcolor_3d = '0xeeeeee'

 

# Generate Conformers

In [4]:
data = [('m1','COc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m2','COc1ccc2[nH]c([S@+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m3','COc1ccc2[nH]c([S+]([O-])Cc3ncc(C)c(OC)c3C)nc2c1'),
       ('m4','CCOc1ccc2[nH]c([S@@+]([O-])Cc3ncc(C)c(OCCC)c3C)nc2c1')]
ms = [(x,Chem.AddHs(Chem.MolFromSmiles(y))) for x,y in data]
params = AllChem.ETKDG()
params.numThreads=3
for nm,m in ms:
    AllChem.EmbedMultipleConfs(m,numConfs=10,params=params)
    # align to one of the ring systems:
    AllChem.AlignMolConformers(m,m.GetSubstructMatch(Chem.MolFromSmarts('c1[nH]c2ccccc2n1')))


# Required functions for scrolling capability

In [5]:

def startViewer(size=None,bgcolor=None):
    if size is None:
        size=molSize_3d
    if bgcolor is None:
        bgcolor=bgcolor_3d
    view = py3Dmol.view(width=size[0],height=size[1])
    view.setBackgroundColor(bgcolor)
    return view


def addMolToViewForScrolling(nm,mols,view,confId,useDrawAs,drawAs):
    
    mol = mols[nm]
    
    view.removeAllModels()
    view.removeAllSurfaces()
    
    if mol.GetNumAtoms()>=999 or drawAs == 'cartoon':
        # py3DMol is happier with TER and MASTER records present
        pdb = Chem.MolToPDBBlock(mol,flavor=0x20|0x10)
        view.addModel(pdb,'pdb')
    else:
        # py3Dmol does not currently support v3k mol files, so
        # we can only provide those with "smaller" molecules
        mb = Chem.MolToMolBlock(mol,confId=confId)
        view.addModel(mb,'sdf')
        
        
    global myGlobalStyle    
    if useDrawAs is False:
        #use from globalStyle
        view.setStyle({},{myGlobalStyle:{}})
    else:
        #update global style and use that
        myGlobalStyle = drawAs
        view.setStyle({},{myGlobalStyle:{}})
    
    # This is exception for surface
    if drawAs is 'surface':
        view.addSurface({}, '$3Dmol.SurfaceType.VDW');
        
    if drawAs is 'ballstick':
        view.setStyle({},{'stick':{'radius':'0.3'},'sphere':{'radius':'0.5'}});
    
    #print(drawAs)
    # zoomTo does not work well for surface
    view.zoomTo()
    return view.update()


def browseMolConformers(ms,view, confId=None, useDrawAs=False, drawAs=None):
    
    drawing_type_3d=['line', 'cross', 'stick', 'cartoon', 'sphere', 'surface', 'ballstick']
    
    global myGlobalStyle
    
    if useDrawAs is False:
        # Then go with default settings
        drawAs = myGlobalStyle
        style_use_logical = widgets.Dropdown(options=[False, True],value=False)
        
        # To do: showing massage is argument mismatched
        # Something like the following
        #if drawAs is not None:
            #print('drawAs set to stick as you choose useDrawAs=False')
            
        
    else:
        style_use_logical = widgets.Dropdown(options=[False, True],value=True)
        # Use user supplied drawAS
        if drawAs is None:
            # User forgot to provide drawAs argument
            drawAs = myGlobalStyle
        else:
            # User supplied drawAs argument while useDrawAs is True
            myGlobalStyle = drawAs
        
    
    # This is wdget for model style
    style_widget = widgets.Dropdown(options=drawing_type_3d,value=drawAs)
    
    moldict = dict(ms)
    
    result=interact(addMolToViewForScrolling, 
                    nm=list(moldict.keys()), 
                    mols=fixed(moldict),
                    view=fixed(view),
                    confId=confId, 
                    useDrawAs=style_use_logical,
                    drawAs=style_widget);
    return result



# Browse mols and confs

In [6]:
view=startViewer(bgcolor='black')
view.show()

In [9]:
browseMolConformers(ms,view,confId=(0,m.GetNumConformers()-1),useDrawAs=True,drawAs='line')

<function __main__.addMolToViewForScrolling>